In [ ]:
'''
CAP6615 - Spring 2022
Assignment #1 - Single- and Multi-Layer Shallow Neural Networks
'''

import numpy as np
import os
import sys
from PIL import Image
import pandas
import matplotlib.pyplot as plt

class performanceMetrics(object):
    def __init__(self, fh, ffa):
        self.fh = fh
        self.ffa = ffa

class Perceptron():
    def __init__(self):
        '''
        Initialize perceptron model.
        '''

    def activation_fn(self,x):
        #     Sigmoid Activation func: f(x) = 1/(1+e^(-x))
        return 1/(1+np.exp(-x))
        
    def _initialize_weights(self):
        '''
        Initialize the weights, initalize using random numbers.
        '''
        self.weights = np.random.randn()

    def predict(self,inputVector, weightMatrix):
        
        predictionVector = []
        for i in range(len(weightMatrix)):
            row_sum = sum(
                (inputVector[j] * weightMatrix[j][i])
                for j in range(len(weightMatrix[0]) - 1)
            )

            predictionVector.append(self.activation_fn(row_sum))
        # print('Prediction')
        # print(predictionVector)   
        return predictionVector

    def calculate_performance_metrics(self, predictionVector, inputVector, outputVector):
        
        totalBlackPixelCount = sum(x == 0 for x in inputVector)
        totalWhitePixelCount = sum(x == 1 for x in inputVector)
        wrongBlackPixelCount = 0
        rightBlackPixelCount = 0
        for i in range(len(outputVector)):
            if outputVector[i] < 0.0001:
                if  abs(predictionVector[i] - inputVector[i]) < 0.0001:
                    rightBlackPixelCount += 1
                else:
                    wrongBlackPixelCount += 1
        fh = rightBlackPixelCount/totalBlackPixelCount
        ffa = wrongBlackPixelCount/totalWhitePixelCount
        return performanceMetrics(fh,ffa)    

    def updateBias(self, predictionVector, outputVector, inputVector, biasVector):
        
        learningRate = 0.8
        for i in range(len(outputVector)):
            error = outputVector[i] - predictionVector[i]
            biasVector[i][0] += (learningRate*inputVector[i]*error) 
        return biasVector


    def fit(self,predictionVector,outputVector,inputVector,weightMatrix):
        
        learningRate = 0.8
        for i in range(len(weightMatrix)):
            error = outputVector[i] - predictionVector[i]
            for j in range(len(weightMatrix[0])):
                weightMatrix[j][i] += (learningRate*inputVector[j]*error) 
        return weightMatrix
    
    def showPredictedImage(self, predictionVector):
        tempArray = [0]*len(predictionVector)
        for i in range(len(predictionVector)):
            if predictionVector[i] < 0.0001:
                tempArray[i] = (0,0,0)
            elif predictionVector[i] == 1.0:
                tempArray[i] =  (255,255,255)
        finalImage = [tempArray[i:i+15] for i in range(0,len(tempArray),16)]

        imageArray = np.array(finalImage, dtype=np.uint8)
        new_image = Image.fromarray(imageArray)
        # new_image.show()



    def main(self):
        tableObject = {}
        plotObject = {}

        for i in range(10):
            tableObject[str(i)+'_fh'] = []
            tableObject[str(i)+'_ffa'] = []
            plotObject[str(i)+'_fh_plot'] = []
            plotObject[str(i)+'_ffa_plot'] = []
    
        for i in range(10):
            path = os.path.join(os.path.dirname(os.path.abspath(sys.argv[1])),'numbers','pixil-frame-'+ str(i) +'.png')
            im = Image.open(path, 'r')
            gray = im.convert('L')
            bw = gray.point(lambda x: 0 if x<135 else 1, '1')
            pix_val_1 = list(bw.getdata())
            weightMatrix = np.random.rand(len(pix_val_1),len(pix_val_1))
            # biasVector = np.random.rand(len(pix_val_1),1)
            epochs = 4
            for _ in range(epochs):
                prediction = self.predict(pix_val_1,weightMatrix)
                weightMatrix = self.fit(prediction,pix_val_1,pix_val_1,weightMatrix)
                # biasVector = self.updateBias(prediction,pix_val_1,pix_val_1,biasVector)      

            for k in range(10):
                testPath = os.path.join(os.path.dirname(os.path.abspath(sys.argv[1])),'numbers','pixil-frame-' + str(k) + '.png')
                testIm = Image.open(testPath, 'r')
                testGray = testIm.convert('L')
                testBW = testGray.point(lambda x: 0 if x<135 else 1, '1')
                testInput = list(testBW.getdata())
                testPrediction = self.predict(testInput,weightMatrix)
                # self.showPredictedImage(testPrediction)
                testMetric = self.calculate_performance_metrics(testPrediction,testInput,testInput)
                tableObject[str(k)+'_fh'].append(round(testMetric.fh,2))
                plotObject[str(i)+'_fh_plot'].append(round(testMetric.fh,2))
                tableObject[str(k)+'_ffa'].append(round(testMetric.ffa,2))
                plotObject[str(i)+'_ffa_plot'].append(round(testMetric.ffa,2))
        
        pandas.DataFrame(tableObject).to_csv('Test_Results.csv')
        for l in range(10):
            plt.scatter(x=plotObject[str(l)+'_ffa_plot'],y=plotObject[str(l)+'_fh_plot'],label='Weights trained on '+str(l),marker="^")
            plt.xlabel('Ffa -->')
            plt.ylabel('Fh -->')
            plt.legend()
            plt.show()    
        
        
if __name__ == "__main__":
    perceptron = Perceptron()
    perceptron.main()

In [ ]:
def metrics(input, output):
    blkPixels = 0.0 # No. of black pixels
    wtPixels = 0.0 # No. of white pixels
    correctPixels = 0.0 # No. of correct (black) pixels
    wrongPixels = 0.0 # No. of wrong (black) pixels
    
    for i in range(len(input)):
        if input[i] == 1: # black = '1'
            blkPixels += 1
        if input[i] == 0: # white = '0'
            wtPixels += 1
        if input[i] == 1 and output[i] == 1: # correct black match
            correctPixels += 1
        if input[i] == 1 and output[i] == 0: # incorrect black/white mismatch
            wrongPixels += 1
    
    Fh = float(correctPixels / blkPixels)
    Ffa = float(wrongPixels / wtPixels)
    
    return Fh, Ffa

In [ ]:
# Training set
    # Dummy values
    a = [0, 1, 1, 0, 1]
    b = [0, 0, 1, 1, 1]
    Fh, Ffa = metrics(a, b)
    plt.scatter(Ffa, Fh, label= "Digits 0-9", marker= "^", color="green", s=30)
    plt.xlabel('Ffa')
    plt.ylabel('Fh')
    plt.title('Fh vs. Ffa for training set')
    plt.show()
    
    
    # Test set (graph)
    # Dummy values
    Fhs = np.random.uniform(low=0.2, high=0.8, size=(90,))
    Ffas = np.random.uniform(low=0.2, high=0.8, size=(90,))
    
    x = np.array([0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 
                  0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002, 0.002,
                  0.003, 0.003, 0.003, 0.003, 0.003, 0.003, 0.003, 0.003, 0.003, 0.003,
                  0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005,
                  0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
                  0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02,
                  0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03,
                  0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05,
                  0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1])
    
    fig, ax = plt.subplots()
    
    ax.scatter(x, Fhs, label="Fh", marker= "^", color="orange", s=30)
    ax.scatter(x, Ffas, label="Ffa", marker= "^", color="red", s=30)
    
    plt.ylim(0, 1)
    ax.set_xscale('log')
    ax.xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
    
    plt.xlabel('Gaussian Noise Level (stdev, at 10 pct xsecn)')
    plt.ylabel("$\it{Fh}$ and $\it{Ffa}$")
    plt.title('Graph of y and y vs. Noise Standard Deviation \n for noise-corrupted Alphanumeric Imagery (16x16 pixels) for \n Autoassociative Single-Layer Perceptron')
    plt.legend()
    plt.show()